In [44]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.optimizers import SGD
from sklearn.preprocessing import OneHotEncoder
from keras.layers import Dense
from keras.regularizers import l2
from tensorflow import keras

In [45]:
lastframe_data= pd.read_csv("../Data/glm_groups.csv")
lastframe_data = lastframe_data.drop(['Unnamed: 0.1','Unnamed: 0', 'rankc','id','game_clock'], axis=1)


In [50]:
def maximum_absolute_scaling(df):
    # copy the dataframe
    df_scaled = df.copy()
    # apply maximum absolute scaling
    for column in df_scaled.columns:
        df_scaled[column] = df_scaled[column]  / df_scaled[column].abs().max()
    return df_scaled
    
# call the maximum_absolute_scaling function
DF_features = maximum_absolute_scaling(lastframe_data)
DF_features = DF_features.drop(columns=['EVENTMSGTYPE'])

In [47]:
y_pred = lastframe_data['EVENTMSGTYPE'].map({2:1,1:0})

In [56]:
model = Sequential()
model.add(Dense(1024, input_dim=3, activation=tf.nn.relu))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(512, activation=tf.nn.relu))
model.add(BatchNormalization())
model.add(Dense(256,activation = tf.nn.relu, kernel_regularizer=keras.regularizers.l2(l=0.1)))
model.add(BatchNormalization())
model.add(Dense(128,activation = tf.nn.relu))
model.add(Dense(1, activation=tf.nn.sigmoid))


In [57]:
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [74]:
tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=0, verbose=0,
    mode='auto', baseline=None, restore_best_weights=True
)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [75]:
x_train, x_test, y_train, y_test = train_test_split(
DF_features, y_pred, test_size=0.1)

In [76]:
x_train, x_val, y_train, y_val = train_test_split(
x_train, y_train, test_size=0.33)

In [77]:
model.fit(x_train, y_train,validation_data = (x_val,y_val), callbacks=[callback], epochs=50, batch_size=64)

Epoch 1/50
195/195 [==============================] - 2s 11ms/step - loss: 0.3568 - accuracy: 0.8491 - val_loss: 0.5120 - val_accuracy: 0.6463
Epoch 2/50
195/195 [==============================] - 2s 11ms/step - loss: 0.3449 - accuracy: 0.8533 - val_loss: 0.2833 - val_accuracy: 0.8887
Epoch 3/50
195/195 [==============================] - 2s 11ms/step - loss: 0.3575 - accuracy: 0.8493 - val_loss: 0.6943 - val_accuracy: 0.6464
Epoch 4/50
195/195 [==============================] - 2s 12ms/step - loss: 0.3483 - accuracy: 0.8531 - val_loss: 0.6894 - val_accuracy: 0.6469
Epoch 5/50
195/195 [==============================] - 2s 10ms/step - loss: 0.3511 - accuracy: 0.8523 - val_loss: 0.4701 - val_accuracy: 0.8049


In [78]:
_, accuracy = model.evaluate(x_test, y_test)

65/65 [==============================] - 0s 2ms/step - loss: 0.4426 - accuracy: 0.8203
